<a href="https://colab.research.google.com/github/PamelaRosa23/projeto-bd-colab/blob/main/Projeto_BD_SQLite_ipynb_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# PROJETO-BD-SQLITE.ipynb
# ---------------------------------------------------------
# Seção 1: Importar bibliotecas e conectar ao BD

import sqlite3

# Conectar ao banco de dados em memória (ou use 'projeto.db' para arquivo)
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

# (Opcional) Garantir que as chaves estrangeiras funcionem
cursor.execute("PRAGMA foreign_keys = ON;")

# ---------------------------------------------------------
# Seção 2: Criação de tabelas

cursor.execute('''
CREATE TABLE IF NOT EXISTS pessoas (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    nome TEXT NOT NULL,
    idade INTEGER
);
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS produtos (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    descricao TEXT NOT NULL,
    estoque INTEGER,
    preco REAL
);
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS vendas (
    id_venda INTEGER PRIMARY KEY AUTOINCREMENT,
    id_pessoa INTEGER,
    id_produto INTEGER,
    quantidade INTEGER,
    FOREIGN KEY (id_pessoa) REFERENCES pessoas(id),
    FOREIGN KEY (id_produto) REFERENCES produtos(id)
);
''')

conn.commit()

# ---------------------------------------------------------
# Seção 3: Inserção de dados

# Inserir pessoas
cursor.executemany("INSERT INTO pessoas (nome, idade) VALUES (?, ?);", [
    ('Maria', 30),
    ('João', 25),
    ('Ana', 40)
])

# Inserir produtos
cursor.executemany("INSERT INTO produtos (descricao, estoque, preco) VALUES (?, ?, ?);", [
    ('Caneta', 100, 1.50),
    ('Caderno', 50, 6.50),
    ('Lápis', 200, 0.75)
])

conn.commit()

# ---------------------------------------------------------
# Seção 4: Exemplo de vendas e consultas

# Inserir vendas
cursor.executemany("INSERT INTO vendas (id_pessoa, id_produto, quantidade) VALUES (?, ?, ?);", [
    (1, 1, 10),  # Maria comprou 10 canetas
    (2, 2, 2),   # João comprou 2 cadernos
    (3, 3, 5)    # Ana comprou 5 lápis
])

conn.commit()

# Consulta com JOIN
print("RELATÓRIO DE VENDAS:")
cursor.execute('''
SELECT v.id_venda, p.nome, pr.descricao, v.quantidade, pr.preco,
       (v.quantidade * pr.preco) AS total
FROM vendas v
JOIN pessoas p ON v.id_pessoa = p.id
JOIN produtos pr ON v.id_produto = pr.id
ORDER BY v.id_venda;
''')

results = cursor.fetchall()

# Exibir resultados formatados
for row in results:
    print(f"Venda #{row[0]} | Cliente: {row[1]} | Produto: {row[2]} | "
          f"Qtd: {row[3]} | Preço: R${row[4]:.2f} | Total: R${row[5]:.2f}")

# ---------------------------------------------------------
# Seção 5: Encerramento

conn.close()


RELATÓRIO DE VENDAS:
Venda #1 | Cliente: Maria | Produto: Caneta | Qtd: 10 | Preço: R$1.50 | Total: R$15.00
Venda #2 | Cliente: João | Produto: Caderno | Qtd: 2 | Preço: R$6.50 | Total: R$13.00
Venda #3 | Cliente: Ana | Produto: Lápis | Qtd: 5 | Preço: R$0.75 | Total: R$3.75
